In [1]:
import os
import re
import csv
import pickle
import pandas as pd
from tqdm import tqdm
from collections import Counter

In [2]:
def read_table(path):
    print('read', path)
    headers = None
    table = []

    fp = open(path, encoding='utf8', errors='ignore')

    for line in tqdm(fp):
        line = line.strip()
        if line:
            if headers is None:
                headers = line.split(',')
                continue

            row = line.split(',')
            if len(row) == len(headers):
                table.append(row)
#     df = pd.DataFrame(table[:20], columns=headers)
#     emotion = []
#     for x in tqdm(table):
#         text = x[6]
#         m = re.findall(r'\[[\u4e00-\u9fff]{1,3}\]', text)
#         if m:
#             emotion += m
#     sample = [
#         x
#         for x in sorted(list(Counter(emotion).items()), key=lambda x: x[1], reverse=True)
#         if x[1] >= 500
#     ]
#     print(len(sample))
#     print(sample[:10])
    return table

In [3]:
def get_labels():
    labels_dict = {
        'happy': [
            '哈哈',
            '偷笑',
            '嘻嘻',
            '耶',
            '太开心',
            '太開心',
            '(^_^)',
            '(*^__^*)',
            '(^o^)',
            '(^.^)',
            'O(∩_∩)O',
        ],
        'sad': [
            '[泪]',
            '[涙]',
            '[傷心]',
            '[悲傷]',
            '[失望]',
            '[伤心]',
            '(T_T)',
            '(T.T)',
            '(T^T)',
            '(ᅲ.ᅲ)',
            '(╯_╰)',
        ],
        'angry': [
            '[怒]',
            '[抓狂]',
            '[哼]',
            '[左哼哼]',
            '[右哼哼]',
            '[怒骂]',
            '[鄙视]',
            '(╰_╯)',
        ],
        'disgust': [
            '[汗]',
            '[晕]',
            '[暈]',
            '[鄙视]', 
            '[鄙視]', 
            '[黑線]', 
            '[囧]', 
            '[黑线]', 
            '[吐]',
        ],
        'fear': [
            '[可怜]', 
            '[生病]', 
            '[委屈]',
        ],
        'suprise': [
            '[吃惊]', 
            '[吃驚]',
            'OMG',
            '(0.o)',
            '(O_o)',
            '(@_@)',
        ],
#         'neutral': [
#             '[酷]', 
#             '[疑问]', 
#             '[握手]', 
#             '[疑問]', 
#             '[可爱]', 
#             '[可愛]', 
#             '[微风]', 
#             '[浮云]', 
#             '[浮雲]', 
#             '[兔子]', 
#             '[思考]', 
#             '[困]',
#         ],
    }
    labels = {}
    for k, v in labels_dict.items():
        for vv in v:
            labels[vv] = k
    
    return labels

In [4]:
LABELS = get_labels()

In [5]:
print(LABELS)

{'(^.^)': 'happy', '(T_T)': 'sad', '[生病]': 'fear', '耶': 'happy', '哈哈': 'happy', '[鄙视]': 'disgust', '(^o^)': 'happy', '(0.o)': 'suprise', '(╰_╯)': 'angry', '[鄙視]': 'disgust', '[晕]': 'disgust', '[吃驚]': 'suprise', '(T.T)': 'sad', 'O(∩_∩)O': 'happy', '太開心': 'happy', '[怒骂]': 'angry', '[抓狂]': 'angry', '[涙]': 'sad', '[悲傷]': 'sad', '太开心': 'happy', '[黑線]': 'disgust', '(T^T)': 'sad', '(ᅲ.ᅲ)': 'sad', '[伤心]': 'sad', '[囧]': 'disgust', '[委屈]': 'fear', '[泪]': 'sad', '[傷心]': 'sad', '(*^__^*)': 'happy', '(^_^)': 'happy', '[哼]': 'angry', '[吃惊]': 'suprise', '嘻嘻': 'happy', '[左哼哼]': 'angry', '[可怜]': 'fear', '(╯_╰)': 'sad', '[黑线]': 'disgust', '偷笑': 'happy', '[汗]': 'disgust', '(@_@)': 'suprise', '[暈]': 'disgust', '[吐]': 'disgust', '[失望]': 'sad', '[怒]': 'angry', '[右哼哼]': 'angry', 'OMG': 'suprise', '(O_o)': 'suprise'}


In [6]:
MIN_LENGTH = 10

def extract_emotion(table):
    emotion_from_table = []
    for x in tqdm(table):
        if len(x) <= 6:
            continue
        text = x[6]
        # 转发取最后一个
        if '//' in text:
            text = text.split('//')[-1]
        # remove at someone
        text = re.sub(r'@[a-zA-Z_0-9\u4e00-\u9fff]+[:：]*(\s+|$)', '', text)
        # remove hash
        text = re.sub(r'#[^#]+#\s*', '', text)
        # without face
        text_without_face = re.sub(r'\s*\[[^\]]+\]\s*', '', text)

        m = re.findall(r'[\u4e00-\u9fff]+', text_without_face)

        if len(m) < MIN_LENGTH:
            continue

        count = 0
        signal_in_text = []
        tag_in_text = None
        for signal, tag in LABELS.items():
            if signal in text:
                if tag == tag_in_text:
                    signal_in_text.append(signal)
                    continue
                elif count >= 1:
                    signal_in_text = None
                    tag_in_text = None
                    break
                else:
                    count += 1
                    signal_in_text.append(signal)
                    tag_in_text = tag

        if tag_in_text is not None:
            clean_text = text_without_face
            for s in signal_in_text:
                clean_text = clean_text.replace(s, '')
            emotion_from_table.append((
                clean_text, text, tag_in_text, signal_in_text))
    return emotion_from_table

In [7]:
root_dir = '/media/qhduan/Seagate Expansion Drive/DATASETS/weibo/'

paths = []
for dirpath, _, filenames in os.walk(root_dir):
    paths += [
        os.path.join(dirpath, x)
        for x in filenames
        if x.endswith('.csv') and 'week' in x
    ]
paths = sorted(paths)

In [8]:
print(len(paths))

52


In [9]:
data = []
for path in paths:
    table = read_table(path)
    emotion_from_table = extract_emotion(table)
    print(len(emotion_from_table))
    data += emotion_from_table

45896it [00:00, 458890.42it/s]

read /media/qhduan/Seagate Expansion Drive/DATASETS/weibo/week1.csv


4790111it [00:15, 302407.98it/s]
100%|██████████| 4633268/4633268 [00:24<00:00, 190433.69it/s]
0it [00:00, ?it/s]

17482
read /media/qhduan/Seagate Expansion Drive/DATASETS/weibo/week10.csv


3882568it [00:11, 325004.26it/s]
100%|██████████| 3752958/3752958 [00:21<00:00, 172780.77it/s]
45870it [00:00, 458643.58it/s]

...

17680
read /media/qhduan/Seagate Expansion Drive/DATASETS/weibo/week8.csv


4739982it [00:11, 430643.67it/s]
100%|██████████| 4595408/4595408 [00:22<00:00, 200984.90it/s]
0it [00:00, ?it/s]

16496
read /media/qhduan/Seagate Expansion Drive/DATASETS/weibo/week9.csv


4010777it [00:14, 269891.76it/s]
100%|██████████| 3878452/3878452 [00:21<00:00, 182475.12it/s]

15708


In [10]:
print(len(data))

880960


In [11]:
with open('data.pkl', 'wb') as fp:
    pickle.dump(data, fp)